In [5]:
import pandas as pd
import numpy as np

# import language_tool_python
# _tool_es = language_tool_python.LanguageTool('es')


from pathlib import Path
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)
from typing import List, Dict
from datetime import datetime
import sys
import os

import re

In [6]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[8]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed

from app.modules.sga.minpub.report_validator.service.objetivos.calculations import has_repetition
from app.modules.sga.minpub.report_validator.service.objetivos.calculations import extract_date_range_body
from app.modules.sga.minpub.report_validator.service.objetivos.calculations import extract_date_range_last


from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_corte_excel
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_335
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_380
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_df_cid_cuismp_sharepoint
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_sga_335_corte_excel_sharepoint_cuismp_sga380 import merge_sga_335_corte_excel_sharepoint_cuismp_sga380


from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_1.o8_medidas_correctivas import validation_medidas_correctivas
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_1.o8_medidas_correctivas import build_failure_messages_medidas_correctivas

In [7]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 2.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_30-03-2025_06-04-2025_20250410_173936.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_30-03-2025_04-04-2025_20250410_195338.xlsx"
DIR_WORD_DATOS = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_datos" / "COMPONENTE 2-DATOS - CORTE 2.docx"
DIR_WORD_TELEFONIA = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_telefonia" / "COMPONENTE 4 - TELEFONOS - CORTE 2.docx"


In [8]:
df_corte_excel = pd.read_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=2, engine="openpyxl")
df_sga_dinamico_335 = pd.read_excel(SAVE_DIR_EXTRACT_SGA_335) 
df_sga_dinamico_380 = pd.read_excel(DIR_PARADAS_RELOJ)
df_cid_cuismp_sharepoint = pd.read_excel(CID_CUISMP_PATH)

In [9]:

df_corte_excel = preprocess_corte_excel(df_corte_excel)
df_cid_cuismp_sharepoint = preprocess_df_cid_cuismp_sharepoint(df_cid_cuismp_sharepoint)
df_sga_dinamico_335 = preprocess_335(df_sga_dinamico_335)
df_sga_dinamico_380 = preprocess_380(df_sga_dinamico_380)

In [10]:
df_matched_corte_sga335_Sharepoint_cuismp_sga380 = merge_sga_335_corte_excel_sharepoint_cuismp_sga380(
        df_corte_excel, df_sga_dinamico_335,
        df_cid_cuismp_sharepoint, df_sga_dinamico_380,
        'both'
        )



In [11]:
df_vali = validation_medidas_correctivas(df_matched_corte_sga335_Sharepoint_cuismp_sga380)

In [12]:
df_mess = build_failure_messages_medidas_correctivas(df_vali)
df_mess

,nro_incidencia,mensaje,TIPO REPORTE,objetivo
7,21790524,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 21/03/2025 15:03 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 27/03/2025 11:30. La fecha de inicio del parrafo en it_medidas_tomadas: ( 19/03/2025 16:22 ) no coincide con la fecha inicio de la penultima fila: 21/03/2025 14:42.,RECLAMO,1.80
15,21791311,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 28/03/2023 10:12 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 21/03/2025 17:40. La fecha de fin del parrafo en it_medidas_tomadas:( 28/03/2023 10:12 ) no coincide con la fecha fin de la ultima fila: 21/03/2025 17:40.,RECLAMO,1.80
35,21795643,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 03/31/2025 08:08 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 31/03/2025 08:08.,PROACTIVO,1.80
36,21796219,La fecha de fin del parrafo en it_medidas_tomadas:( 31/03/2025 06:30 ) no coincide con la fecha fin de la ultima fila: 31/03/2025 06:30.,PROACTIVO,1.80
40,21796282,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 15/03/2025 15:30 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 31/03/2025 15:30. La fecha de fin del parrafo en it_medidas_tomadas:( 15/03/2025 15:30 ) no coincide con la fecha fin de la ultima fila: 31/03/2025 15:30.,RECLAMO,1.80
41,21796323,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 31/03/2025 16:00 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 31/03/2025 17:54. La fecha de inicio del parrafo en it_medidas_tomadas: ( 31/03/2025 10:12 ) no coincide con la fecha inicio de la penultima fila: None. La fecha de fin del parrafo en it_medidas_tomadas:( 31/03/2025 16:00 ) no coincide con la fecha fin de la ultima fila: None.,RECLAMO,1.80
43,21796489,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 01/04/2025 11:09 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 01/04/2025 11:06. La fecha de fin del parrafo en it_medidas_tomadas:( 01/04/2025 11:09 ) no coincide con la fecha fin de la ultima fila: 01/04/2025 11:09.,PROACTIVO,1.80
44,21796518,La fecha de inicio del parrafo en it_medidas_tomadas: ( 31/03/2025 14:49 ) no coincide con la fecha inicio de la penultima fila: 32/03/2025 14:49. La fecha de fin del parrafo en it_medidas_tomadas:( 31/03/2025 16:43 ) no coincide con la fecha fin de la ultima fila: 32/03/2025 16:43.,PROACTIVO,1.80
46,21796590,La fecha de inicio del parrafo en it_medidas_tomadas: ( 31/03/2025 15:49 ) no coincide con la fecha inicio de la penultima fila: None. La fecha de fin del parrafo en it_medidas_tomadas:( 31/03/2025 17:33 ) no coincide con la fecha fin de la ultima fila: None.,RECLAMO,1.80
47,21796622,La fecha/hora de fin del parrafo en MEDIDAS CORRECTIVAS:( 01/03/2025 10:15 ) no coincide con la columna FECHA Y HORA FIN DE EXCEL: 01/04/2025 10:15.,RECLAMO,1.80
